In [2]:
import os
import pandas as pd
from langchain.llms.openai import OpenAI
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv() #Take environment variables from .env 


True

In [35]:
KEY=os.getenv("OPENAI_API_KEY_2")

In [36]:
llm=ChatOpenAI(openai_api_key=KEY, model_name = "gpt-3.5-turbo", temperature=0.5)

In [37]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [38]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions
for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs
{responses_json}
"""

In [39]:
RESPONSE_JSON = {
    "1": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3": {
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "responses_json"],
    template=TEMPLATE
)

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose = True)

/tmp/ipykernel_1266/1059869629.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose = True)


In [14]:
TEMPLATE_2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert in English Writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE_2)


In [16]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True )

In [17]:
generate_evaluation_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "responses_json"],
                                          output_variables=["quiz", "review"], verbose=True)

In [21]:
file_path="/home/gauva/mcqgen/data.txt"

In [22]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [23]:
TEXT

"In today’s rapidly evolving society, the conversation around gender equality in the workplace cannot be overlooked or overstated. \n\nWhile there has undoubtedly been considerable progress compared to previous generations, we are still far from having established gender parity in the workplace, and the issue of inequality persists. \n\nSubsequently, the conversation around gender inequality must remain ongoing and proactive measures must be taken to address gender inequality concerns and champion women's empowerment at work.\n\nChange can only be achieved through unity, where everyone, regardless of gender, understands the impropriety of gender bias and diligently work towards combating it, to ensure equal rights and opportunities for all. \n\nMost importantly, employers must take steps to promote and set an example of what gender equality in the workplace should look like. \nWhat is Gender Equality?\n\n Defining the true meaning of gender equality is the first step towards promoting 

In [25]:
#Serialize Python dictionary into a JSON-formatted string
import json
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [26]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [40]:
#Token usage tracking
#How to setup Token usage Tracking in Langchain
#Use this to calculate the token and the cost of input and generated output tokens through OPEN AI API
# No money -> No result. Sr. This is how OpenAI works 
with get_openai_callback() as cb:
    response=generate_evaluation_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "responses_json": json.dumps(RESPONSE_JSON)
        }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:In today’s rapidly evolving society, the conversation around gender equality in the workplace cannot be overlooked or overstated. 

While there has undoubtedly been considerable progress compared to previous generations, we are still far from having established gender parity in the workplace, and the issue of inequality persists. 

Subsequently, the conversation around gender inequality must remain ongoing and proactive measures must be taken to address gender inequality concerns and champion women's empowerment at work.

Change can only be achieved through unity, where everyone, regardless of gender, understands the impropriety of gender bias and diligently work towards combating it, to ensure equal rights and opportunities for all. 

Most importantly, employers must take steps to promote and set an example of what gender equality in the workplace should look like. 
What is Gender Equality?

 Defining the true meaning of gender equality is the first step

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [41]:
print(f"Total Tokens:{cb.total_tokens}")#2452
print(f"Prompt Tokens:{cb.prompt_tokens}")#1808
print(f"Completion Tokens:{cb.completion_tokens}")#644
print(f"Total Cost:{cb.total_cost}") # 0,004

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [42]:
response #print the response from OpenAI

NameError: name 'response' is not defined

In [ ]:
quiz=response.get("quiz") #Print out the quiz in json

In [ ]:
quiz=json.loads(quiz) #Print out the whole generated Quizs + Correct answers

In [47]:
#extract the question, options and correct answers
#options are in this format 'a: ... | b: ... | c: ... | d: ...'
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct =value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})

NameError: name 'quiz' is not defined

In [ ]:
quiz=pd.DataFrame(quiz_table_data) #Data in panda form - 3 Columns "MCQ", "Choices", "Correct"

""


In [ ]:
quiz.to_csv("maschinelearning.csv", index=False) #generate a .csv file and the quiz within